In [ ]:
import os
import sys
import pandas as pd
import numpy as np

module_path = os.path.abspath(os.path.join('..\..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from src import helpers

In [ ]:
activities = os.listdir('..\\..\\data\\activities')
activities = list(set(activities))

In [ ]:
first_activity = activities[0]
path = '..\\..\\data\\activities\\{}'.format(first_activity)
df = pd.read_csv(path, encoding='latin1')
df["Id"] = activities[0][:-4]
df = df.set_index(["Id", "Intervalle"])

for activity in activities[1:]:
    path = '..\\..\\data\\activities\\{}'.format(activity)
    df1 = pd.read_csv(path, encoding='latin1')
    df1["Id"] = activity[:-4]
    if len(df1) != 0:
        df1 = df1.set_index(["Id", "Intervalle"])
        df = df1.append(df)
    else:
        print("Empty Dataframe for activity {}.csv".format(activity))

## Delete rows without "Temps de déplacement", "Distance" or "Pace"

In [ ]:
df["Distance"].replace('--', np.nan, inplace=True)
df["Temps de déplacement"].replace('--', np.nan, inplace=True)
df["Allure moyenne en déplacement"].replace('--', np.nan, inplace=True)

df.dropna(inplace=True)

## Bugs on "Temps de déplacement"

In [ ]:
# As for
df.loc[("1422118663", "1")]

## Column types and subsetting

In [ ]:
df["pace"] = df.apply(helpers.get_mps, axis=1)
df["best_pace"] = df.apply(helpers.get_best_mps, axis=1)
df["time"] = df.apply(helpers.min_seconds, axis=1)
df["distance"] = pd.to_numeric(df["Distance"], errors="coerce")
df["d_plus"] = pd.to_numeric(df["Gain d'altitude"], errors="coerce").fillna(0).astype(int)
df["d_minus"] = pd.to_numeric(df["Perte d'altitude"], errors="coerce").fillna(0).astype(int)
df["cadence"] = pd.to_numeric(df["Cadence de course moyenne"], errors="coerce")
df["stride_length"] = pd.to_numeric(df["Longueur moyenne des foulées"], errors="coerce")
df["avg_hr"] = pd.to_numeric(df["Fréquence cardiaque moy."], errors="coerce")
df["max_hr"] = pd.to_numeric(df["Fréquence cardiaque maximale"], errors="coerce")
df["kcal"] = pd.to_numeric(df["Calories"], errors="coerce").fillna(0).astype(int)
df = df.loc[:, ["distance", "time", "pace", "best_pace", "d_plus", "d_minus", "cadence", "stride_length", "avg_hr", "max_hr", "kcal"]]
df

In [ ]:
df.sort_values("time", ascending=False)

## Exporting as CSV

In [ ]:
df.to_csv('..\\..\\data\\processed\\laps.csv')